![cyber_photo](cyber_photo.jpg)

Cyber threats are a growing concern for organizations worldwide. These threats take many forms, including malware, phishing, and denial-of-service (DOS) attacks, compromising sensitive information and disrupting operations. The increasing sophistication and frequency of these attacks make it imperative for organizations to adopt advanced security measures. Traditional threat detection methods often fall short due to their inability to adapt to new and evolving threats. This is where deep learning models come into play.

Deep learning models can analyze vast amounts of data and identify patterns that may not be immediately obvious to human analysts. By leveraging these models, organizations can proactively detect and mitigate cyber threats, safeguarding their sensitive information and ensuring operational continuity.

As a cybersecurity analyst, you identify and mitigate these threats. In this project, you will design and implement a deep learning model to detect cyber threats. The BETH dataset simulates real-world logs, providing a rich source of information for training and testing your model. The data has already undergone preprocessing, and we have a target label, `sus_label`, indicating whether an event is malicious (1) or benign (0).

By successfully developing this model, you will contribute to enhancing cybersecurity measures and protecting organizations from potentially devastating cyber attacks.


### The Data

| Column     | Description              |
|------------|--------------------------|
|`processId`|The unique identifier for the process that generated the event - int64 |
|`threadId`|ID for the thread spawning the log - int64|
|`parentProcessId`|Label for the process spawning this log - int64|
|`userId`|ID of user spawning the log|Numerical - int64|
|`mountNamespace`|Mounting restrictions the process log works within - int64|
|`argsNum`|Number of arguments passed to the event - int64|
|`returnValue`|Value returned from the event log (usually 0) - int64|
|`sus_label`|Binary label as suspicous event (1 is suspicious, 0 is not) - int64|

More information on the dataset: [BETH dataset](accreditation.md)

In [15]:
# Make sure to run this cell to use torchmetrics. If you cannot use pip install to install the torchmetrics, you can use sklearn.
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [16]:
# Import required libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
from torch.nn import functional as F
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torchmetrics import Accuracy
# from sklearn.metrics import accuracy_score  # uncomment to use sklearn

In [17]:
# Load preprocessed data
train_df = pd.read_csv('labelled_train.csv')
test_df = pd.read_csv('labelled_test.csv')
val_df = pd.read_csv('labelled_validation.csv')

# View the first 5 rows of training set
train_df.head()

,processId,threadId,parentProcessId,userId,mountNamespace,argsNum,returnValue,sus_label
0,381,7337,1,100,4026532231,5,0,1
1,381,7337,1,100,4026532231,1,0,1
2,381,7337,1,100,4026532231,0,0,1
3,7347,7347,7341,0,4026531840,2,-2,1
4,7347,7347,7341,0,4026531840,4,0,1


In [18]:
# Data Preprocessing
def prepare_data(df):
    # Select feature columns (all except sus_label)
    features = ['processId', 'threadId', 'parentProcessId', 'userId', 
                'mountNamespace', 'argsNum', 'returnValue']
    X = df[features].values
    y = df['sus_label'].values
    return X, y

# Scale the features
scaler = StandardScaler()
X_train, y_train = prepare_data(train_df)
X_val, y_val = prepare_data(val_df)
X_test, y_test = prepare_data(test_df)

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.LongTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [19]:
# Define the neural network
class CyberAttackDetector(nn.Module):
    def __init__(self, input_size):
        super(CyberAttackDetector, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # 2 classes: normal (0) and suspicious (1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [20]:
# Initialize model, loss function, and optimizer
input_size = X_train_scaled.shape[1]
model = CyberAttackDetector(input_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
accuracy_metric = Accuracy(task="multiclass", num_classes=2)

In [21]:
# Training loop
num_epochs = 10
val_accuracy = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_predictions = []
    val_labels = []
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            val_predictions.extend(predicted.tolist())
            val_labels.extend(labels.tolist())
    
    # Calculate validation accuracy
    val_accuracy = accuracy_metric(
        torch.tensor(val_predictions), 
        torch.tensor(val_labels)
    ).item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss/len(train_loader):.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}')

# Convert validation accuracy to integer percentage
val_accuracy = int(val_accuracy * 100)
print(f"\nFinal Validation Accuracy: {val_accuracy}%")

Epoch [1/10], Loss: 0.0039, Validation Accuracy: 1.0000
Epoch [2/10], Loss: 0.0022, Validation Accuracy: 1.0000
Epoch [3/10], Loss: 0.0023, Validation Accuracy: 1.0000
Epoch [4/10], Loss: 0.0021, Validation Accuracy: 1.0000
Epoch [5/10], Loss: 0.0021, Validation Accuracy: 1.0000
Epoch [6/10], Loss: 0.0019, Validation Accuracy: 1.0000
Epoch [7/10], Loss: 0.0021, Validation Accuracy: 1.0000
Epoch [8/10], Loss: 0.0020, Validation Accuracy: 1.0000
Epoch [9/10], Loss: 0.0020, Validation Accuracy: 1.0000
Epoch [10/10], Loss: 0.0020, Validation Accuracy: 1.0000

Final Validation Accuracy: 99%
